In [23]:
import pandas as pd
import catboost as cb
import numpy as np
import warnings
import gc
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, cohen_kappa_score, make_scorer, f1_score, recall_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
warnings.simplefilter('ignore')
import pprint, pydotplus
from pylab import rcParams
import optuna.integration.lightgbm as lgb
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submit_df = pd.read_csv('sample_submit.csv',header=None)
train_df.shape,test_df.shape

((55583, 29), (18528, 28))

In [3]:
test_df['y']= -999
All_df = pd.concat([train_df,test_df])
All_df.shape

(74111, 29)

In [4]:
all_df = All_df.drop(['amenities','description','host_has_profile_pic', \
               'host_identity_verified','host_response_rate',\
               'host_since', 'last_review', 'last_review','name',\
               'first_review','neighbourhood','thumbnail_url','zipcode'], axis=1)

In [5]:
# all_df['zipcode'] = all_df['zipcode'].fillna(-1).astype(str)

In [6]:
All_df['city'].unique()

array(['LA', 'DC', 'NYC', 'SF', 'Chicago', 'Boston'], dtype=object)

In [7]:
us_city = pd.read_csv('uscities.csv',usecols = ['state_id','lat','lng','population'],dtype = {'population': int}).rename(columns = {'state_id':'city'})

In [8]:
city = us_city.sort_values('population',ascending = False).head(30)

In [9]:
def cal_rho(lon_a,lat_a,lon_b,lat_b):
    ra=6378.140  # equatorial radius (km)
    rb=6356.755  # polar radius (km)
    F=(ra-rb)/ra # flattening of the earth
    rad_lat_a=np.radians(lat_a)
    rad_lon_a=np.radians(lon_a)
    rad_lat_b=np.radians(lat_b)
    rad_lon_b=np.radians(lon_b)
    pa=np.arctan(rb/ra*np.tan(rad_lat_a))
    pb=np.arctan(rb/ra*np.tan(rad_lat_b))
    xx=np.arccos(np.sin(pa)*np.sin(pb)+np.cos(pa)*np.cos(pb)*np.cos(rad_lon_a-rad_lon_b))
    c1=(np.sin(xx)-xx)*(np.sin(pa)+np.sin(pb))**2/np.cos(xx/2)**2
    c2=(np.sin(xx)+xx)*(np.sin(pa)-np.sin(pb))**2/np.sin(xx/2)**2
    dr=F/8*(c1-c2)
    rho=ra*(xx+dr)
    return rho

In [10]:
all_df.iat[0,10]

33.788930644383385

In [11]:
all_df.head()

,id,accommodates,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,instant_bookable,latitude,longitude,number_of_reviews,property_type,review_scores_rating,room_type,y
0,0,6,2.0,Real Bed,1.0,4.0,flexible,LA,t,f,33.788931,-118.154761,1,Apartment,60.0,Private room,138.0
1,1,2,1.0,Real Bed,1.0,1.0,strict,DC,t,f,38.934810,-76.978190,9,House,100.0,Private room,42.0
2,2,2,2.0,Real Bed,1.0,1.0,strict,NYC,t,t,40.695118,-73.926240,27,Apartment,83.0,Private room,65.0
3,3,2,1.0,Real Bed,1.0,1.0,strict,SF,t,t,37.796728,-122.411906,38,Apartment,95.0,Private room,166.0
4,4,2,1.0,Real Bed,1.0,1.0,strict,NYC,t,f,40.785050,-73.974691,5,Apartment,100.0,Entire home/apt,165.0


In [12]:
dist = [[] for _ in range(all_df.shape[0])]
temp = [[] for _ in range(city.shape[0])]
city_n = [[] for _ in range(all_df.shape[0])]

for n in range(all_df.shape[0]):
    for i in range(city.shape[0]):
        temp[i] = cal_rho(all_df.iat[n,12],all_df.iat[n,11],city.iat[i,2],city.iat[i,1])
    city_n[n] = city.iat[temp.index(max(temp)),3]
    dist[n] = min(temp)

In [13]:
all_df['dist'] = dist
all_df['city'] = city_n
all_df.head(50)

,id,accommodates,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,instant_bookable,latitude,longitude,number_of_reviews,property_type,review_scores_rating,room_type,y,dist
0,0,6,2.0,Real Bed,1.0,4.0,flexible,3210314,t,f,33.788931,-118.154761,1,Apartment,60.0,Private room,138.0,5072.921538
1,1,2,1.0,Real Bed,1.0,1.0,strict,3643765,t,f,38.934810,-76.978190,9,House,100.0,Private room,42.0,12756.961815
2,2,2,2.0,Real Bed,1.0,1.0,strict,3643765,t,t,40.695118,-73.926240,27,Apartment,83.0,Private room,65.0,13266.428190
3,3,2,1.0,Real Bed,1.0,1.0,strict,3210314,t,t,37.796728,-122.411906,38,Apartment,95.0,Private room,166.0,7133.949248
4,4,2,1.0,Real Bed,1.0,1.0,strict,3643765,t,f,40.785050,-73.974691,5,Apartment,100.0,Entire home/apt,165.0,12591.453069
5,5,2,1.0,Real Bed,1.0,1.0,flexible,3643765,t,f,40.640241,-74.015729,7,House,94.0,Private room,55.0,12652.762417
6,6,2,1.5,Real Bed,1.0,1.0,moderate,3643765,t,t,40.676824,-73.915965,65,House,91.0,Private room,48.0,14278.064677
7,7,2,1.0,Pull-out Sofa,1.0,1.0,flexible,3210314,f,f,34.068441,-118.353515,0,Apartment,NaN,Private room,150.0,5024.307036
8,8,2,1.0,Real Bed,1.0,1.0,strict,3210314,t,f,37.801514,-122.411410,21,Apartment,87.0,Private room,95.0,6284.853230
9,9,2,1.0,Real Bed,1.0,1.0,moderate,3643765,t,f,40.751933,-73.878733,27,House,88.0,Private room,79.0,13267.262190


カテゴリカルデータをint型に変換

In [14]:
categorical_features = ['bed_type', 'cancellation_policy', 'city', 'cleaning_fee',\
                        'instant_bookable', 'property_type','room_type']
for col in categorical_features:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(all_df[col])
    lbl.transform(all_df[col])
    all_df[col]=lbl.transform(all_df[col])
    
# MultiLabel= ['amenities']
# for col in MultiLabel:
#     lbl = preprocessing.MultiLabelBinarizer()
#     lbl.fit(all_df[col])
#     lbl.transform(all_df[col])
#     all_df[col]=lbl.transform(all_df[col])

In [15]:
all_df.head(10)

,id,accommodates,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,instant_bookable,latitude,longitude,number_of_reviews,property_type,review_scores_rating,room_type,y,dist
0,0,6,2.0,4,1.0,4.0,0,2,1,0,33.788931,-118.154761,1,0,60.0,1,138.0,5072.921538
1,1,2,1.0,4,1.0,1.0,2,3,1,0,38.934810,-76.978190,9,17,100.0,1,42.0,12756.961815
2,2,2,2.0,4,1.0,1.0,2,3,1,1,40.695118,-73.926240,27,0,83.0,1,65.0,13266.428190
3,3,2,1.0,4,1.0,1.0,2,2,1,1,37.796728,-122.411906,38,0,95.0,1,166.0,7133.949248
4,4,2,1.0,4,1.0,1.0,2,3,1,0,40.785050,-73.974691,5,0,100.0,0,165.0,12591.453069
5,5,2,1.0,4,1.0,1.0,0,3,1,0,40.640241,-74.015729,7,17,94.0,1,55.0,12652.762417
6,6,2,1.5,4,1.0,1.0,1,3,1,1,40.676824,-73.915965,65,17,91.0,1,48.0,14278.064677
7,7,2,1.0,3,1.0,1.0,0,2,0,0,34.068441,-118.353515,0,0,NaN,1,150.0,5024.307036
8,8,2,1.0,4,1.0,1.0,2,2,1,0,37.801514,-122.411410,21,0,87.0,1,95.0,6284.853230
9,9,2,1.0,4,1.0,1.0,1,3,1,0,40.751933,-73.878733,27,17,88.0,1,79.0,13267.262190


In [16]:
# 訓練データ、テストデータの分割
train_df = all_df[all_df['y']!= -999]
test_df = all_df[all_df['y'] == -999]

In [17]:
test_df.shape

(18528, 18)

In [18]:
# train_df.loc[:,'bed_type']

In [19]:
y_train = train_df['y']
X_train = train_df.drop(['y','id'], axis=1)
X_test = test_df.drop(['y','id'], axis=1)

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

In [21]:
X_train.shape,X_valid.shape,y_train.shape, y_valid.shape

((38908, 16), (16675, 16), (38908,), (16675,))

In [24]:
# 使用モデルはLGB（パラメータチューニング無）
train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features,free_raw_data=False)
test = lgb.Dataset(X_valid, y_valid, reference=train, categorical_feature=categorical_features,free_raw_data=False)

params = {
    'objective': 'regression',
    'metric': 'rmse'
}

model = lgb.train(
    params, train,
    valid_sets=[train, test],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 124.118	valid_1's rmse: 127.655
[20]	training's rmse: 110.987	valid_1's rmse: 116.814
[30]	training's rmse: 105.59	valid_1's rmse: 113.341
[40]	training's rmse: 102.414	valid_1's rmse: 111.648
[50]	training's rmse: 100.491	valid_1's rmse: 110.946
[60]	training's rmse: 99.1317	valid_1's rmse: 110.673
[70]	training's rmse: 98.149	valid_1's rmse: 110.574
[80]	training's rmse: 97.215	valid_1's rmse: 110.417
[90]	training's rmse: 96.4336	valid_1's rmse: 110.336
[100]	training's rmse: 95.5936	valid_1's rmse: 110.283
[110]	training's rmse: 94.8549	valid_1's rmse: 110.265
Early stopping, best iteration is:
[106]	training's rmse: 95.1897	valid_1's rmse: 110.248


feature_fraction, val_score: 110.248091:  14%|######1                                    | 1/7 [00:00<00:01,  3.68it/s]

Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 122.721	valid_1's rmse: 126.433
[20]	training's rmse: 110.397	valid_1's rmse: 116.421
[30]	training's rmse: 105.08	valid_1's rmse: 113.101
[40]	training's rmse: 101.986	valid_1's rmse: 111.601
[50]	training's rmse: 100.135	valid_1's rmse: 110.942
[60]	training's rmse: 98.6563	valid_1's rmse: 110.586
[70]	training's rmse: 97.5268	valid_1's rmse: 110.41
[80]	training's rmse: 96.5394	valid_1's rmse: 110.378
Early stopping, best iteration is:
[76]	training's rmse: 96.9059	valid_1's rmse: 110.325


feature_fraction, val_score: 110.248091:  29%|############2                              | 2/7 [00:00<00:01,  4.09it/s]

...

min_data_in_leaf, val_score: 109.313121:  60%|#########################8                 | 3/5 [00:00<00:00,  4.16it/s]

Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 124.56	valid_1's rmse: 128.704
[20]	training's rmse: 111.21	valid_1's rmse: 117.465
[30]	training's rmse: 105.025	valid_1's rmse: 113.737
[40]	training's rmse: 101.603	valid_1's rmse: 112.432
[50]	training's rmse: 99.128	valid_1's rmse: 111.485
[60]	training's rmse: 97.3948	valid_1's rmse: 111.145
[70]	training's rmse: 95.8064	valid_1's rmse: 110.834
[80]	training's rmse: 94.6221	valid_1's rmse: 110.608
[90]	training's rmse: 93.5213	valid_1's rmse: 110.645
Early stopping, best iteration is:
[88]	training's rmse: 93.741	valid_1's rmse: 110.593


min_data_in_leaf, val_score: 109.313121:  80%|##################################4        | 4/5 [00:00<00:00,  4.35it/s]

Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 125.138	valid_1's rmse: 128.6
[20]	training's rmse: 112.349	valid_1's rmse: 117.171
[30]	training's rmse: 106.973	valid_1's rmse: 113.518
[40]	training's rmse: 104.167	valid_1's rmse: 111.786
[50]	training's rmse: 102.121	valid_1's rmse: 110.925
[60]	training's rmse: 100.805	valid_1's rmse: 110.473
[70]	training's rmse: 99.7562	valid_1's rmse: 110.21
[80]	training's rmse: 98.8548	valid_1's rmse: 110.041
[90]	training's rmse: 98.1369	valid_1's rmse: 109.903
[100]	training's rmse: 97.5163	valid_1's rmse: 109.755
[110]	training's rmse: 96.9186	valid_1's rmse: 109.742
Early stopping, best iteration is:
[106]	training's rmse: 97.1485	valid_1's rmse: 109.72


min_data_in_leaf, val_score: 109.313121: 100%|###########################################| 5/5 [00:01<00:00,  4.28it/s]


In [25]:
submit_df[1]=y_pred
submit_df.to_csv('submit7.csv',index=False,header=None)